<a href="https://colab.research.google.com/github/RobNova1/test-colab/blob/master/colab_clavis_validation_20190510.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CLAVIS VALIDATION - PRICE PROMOTIONS

- Using the Clavis - Price Promotions file downloaded from CLAVIS SITE, NOT AUTOMATED SCRIPT
- Report Date: 2019-05-10

# For Reference:
___

 <img src='https://drive.google.com/open?id=1BCHiW_LZwZRVOhfVa91uLEeQDAXoyey7' width='600'/>
___

content/sample_data/california_housing_train.csv

content/prpromo-2019-05-10.png

** Import pandas as pd.**

In [0]:
import pandas as pd

In [1]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

NameError: ignored

In [0]:
from google.colab import drive
drive.mount('/content/drive')

** Read Price_-_Products-2019-05-10.csv as a dataframe called prpromo.**

In [0]:
#prpromo = pd.read_csv('Price_-_Products-2019-05-10.csv')

path = "/content/drive/My Drive/colab-data/Price_-_Products-2019-05-10.csv"
prpromo = pd.read_csv(path)

# Dataset is now stored in a Pandas Dataframe

FileNotFoundError: ignored

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
with open('/content/gdrive/My Drive/foo.txt', 'w') as f:
  f.write('Hello Google Drive!')
!cat /content/gdrive/My\ Drive/foo.txt

Hello Google Drive!

In [0]:
with open('/content/gdrive/My Drive/Colab Notebooks/clavis-validation/Price_-_Products-2019-05-10.csv', 'w') as f:
  #f.write('Hello Google Drive!')
  !cat /content/gdrive/My\ Drive/Colab\ Notebooks/clavis-validation/Price_-_Products-2019-05-10.csv

** Check the head of the DataFrame. **

** Use the .info() method to find out how many entries there are.**

In [0]:
prpromo.info() # 148654 Entries

In [0]:
#prpromo.groupby('online_store')['net_price'].sum()

In [0]:
df = prpromo[['trusted_rpc','trusted_product_description', 'online_store', 'net_price', 'discount_percent']]

# Test a few products

In [0]:
df[df['trusted_product_description'].str.contains('Febreze')]

In [0]:
df[df['trusted_product_description'].str.contains('360')].head()

In [0]:
#prpromo[prpromo.discount_amount > 0].count()

In [0]:
#prpromo['promotion_type'].unique()

In [0]:
#prpromo['promotion_type'].str.match('Uncategorized').count()

 # create concat field, to drop duplicate values from

In [0]:
prpromo['concat']=prpromo['trusted_rpc']+prpromo['online_store']


In [0]:
prpromo.drop_duplicates('concat', inplace=True)

In [0]:
df = prpromo[['concat','trusted_rpc','trusted_product_description', 'online_store', 'net_price', 'discount_percent']]

In [0]:
df.count()

In [0]:
df['concat'].nunique()

In [0]:
df.sort_values('discount_percent', ascending=False).head(10)

# BQ Validation

Comparing for same day as above


In [0]:
from google.cloud import bigquery
client = bigquery.Client()

ModuleNotFoundError: No module named 'google'

In [0]:
sql = """
WITH
  PrPromoConcat AS (
  SELECT
    CONCAT(trusted_rpc, online_store) AS concat,
    trusted_rpc,
    trusted_product_description,
    online_store,
    net_price,
    discount_percent
  FROM
    `cp-gaa-visualization-dev.clavis.pricing_promotions`
  WHERE
    report_date = "2019-05-10"
  LIMIT
    1000 ),
  PrPromoCTE AS (
  SELECT
    *,
    ROW_NUMBER()OVER(PARTITION BY concat ORDER BY concat) AS RowNumber
  FROM
    PrPromoConcat )
SELECT
  trusted_rpc,
  trusted_product_description,
  online_store,
  net_price,
  discount_percent
FROM
  PrPromoCTE
WHERE
  RowNumber < 2
ORDER BY
  discount_percent DESC
"""
dfbq = client.query(sql).to_dataframe()
dfbq.head()

,trusted_rpc,trusted_product_description,online_store,net_price,discount_percent
0,973249,Dove Conditioner Coconut & Hydration,Walgreens,0.3,93.74
1,75661329,Oral-B Black Genius 9600 Rechargeable Electric...,Target,20,90.00
2,75661331,Oral-B Sakura Pink Genius 9600 Rechargeable El...,Target,20,90.00
3,75661335,Oral-B Rose Gold Genius 9600 Rechargeable Elec...,Target,20,90.00
4,75662986,Oral-B 9600 Orchid Purple Electric Toothbrush ...,Target,20,90.00


,trusted_rpc,trusted_product_description,online_store,net_price,discount_percent
0,973249,Dove Conditioner Coconut & Hydration,Walgreens,0.3,93.74
1,75661329,Oral-B Black Genius 9600 Rechargeable Electric...,Target,20,90.00
2,75661331,Oral-B Sakura Pink Genius 9600 Rechargeable El...,Target,20,90.00
3,75661335,Oral-B Rose Gold Genius 9600 Rechargeable Elec...,Target,20,90.00
4,75662986,Oral-B 9600 Orchid Purple Electric Toothbrush ...,Target,20,90.00


In [0]:
dfbq.count()

trusted_rpc                    771
trusted_product_description    771
online_store                   771
net_price                      771
discount_percent               771
dtype: int64

trusted_rpc                    771
trusted_product_description    771
online_store                   771
net_price                      771
discount_percent               771
dtype: int64